# The R workflow of Anne
---------------------------------------

In this jupyter-notebook, I'll show you the R workflow of Anne as the sample.

You can understand the basic workflow by reading this, but the RCy3's documentation is really useful and you can know much more about this.

The documentation : https://www.bioconductor.org/packages/release/bioc/html/RCy3.html

# Setup
---------------------------------------
To execute this, please sutisfy following items.

- Java SE 8
- Cytoscape version 3.3+
- CyREST
- Use docker file to do this. The docker file provide you the environment below.
    - RCy3
    - igraph
    - ...

# Start a new session
---------------------------------------
To start a new session, you should do the following step.

### Steps
1. Please execute cytoscape
1. Import libraries
1. Load Data
1. Shaping data
1. ...

In [7]:
library(RCy3)
library(igraph)

In [8]:
# Load Data
gal.table <- read.table('sampleData/galFiltered.sif',stringsAsFactors=FALSE)
head(gal.table)

V1,V2,V3
YKR026C,pp,YGL122C
YGR218W,pp,YGL097W
YGL097W,pp,YOR204W
YLR249W,pp,YPR080W
YLR249W,pp,YBR118W
YLR293C,pp,YGL097W


In [9]:
# create graph class
g <- new ('graphNEL', edgemode='directed')

# Get NodesVec
gal.table.nodevec <- unique(c(gal.table[[1]], gal.table[[3]]))

# add nodes to graph
for(node in gal.table.nodevec){
    g <- graph::addNode(node, g)
}

# get EdgeList
gal.table.fromvec = gal.table[[1]]
gal.table.tovec = gal.table[[3]]

for (index in 1:length(gal.table.fromvec)){
    g <- graph::addEdge (gal.table.fromvec[[index]] ,gal.table.tovec[[index]], g)
}

# show it in cytescape
cw <- CytoscapeWindow('vignette', , graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')


Warning message:
In .local(from, to, graph): edges replaced: ‘YPL248C|YML051W’

[1] "label"


Then, you can get this image in cytoscape.

![cytoscape image](graph_image.jpeg)

# Upload  a node attribute table.

---------------------------------------

## Import Annotation Data from bioconductor and merge it into Network data
---------------------------------------

In this example of workflow, by executing the following steps, you can load annotation data and merge it.

1. First, import network data from local file/URL to Cytoscape.
1. Second, import annotation data from database(in R, we can use bioconductor as database).
  - Human database in bioconductor:
  - https://www.bioconductor.org/packages/release/data/annotation/html/org.Hs.eg.db.html
  - Yeast database in bioconductor :
  - https://www.bioconductor.org/packages/release/data/annotation/html/org.Sc.sgd.db.html
1. Finally, merge the above two data table and push it to Cytoscape.

Now, we use the data file 'galFiltered.sif' that is the yeast data. So, we will try to add annotation data to this.


In [10]:
# Second, import annotation data from database(in R, we can use bioconductor as database).
# import library to access database
library(org.Sc.sgd.db)

# import Data
# DESCRIPTION data
descriptions <- select(org.Sc.sgd.db, keys=getAllNodes(cw), columns="DESCRIPTION")
df.descriptions <- data.frame(descriptions)

# GENENAME data
gene.names <- select(org.Sc.sgd.db, keys=getAllNodes(cw), columns="GENENAME")
df.gene.names <- data.frame(gene.names)

# GO data
go <- select(org.Sc.sgd.db, keys=getAllNodes(cw), columns="GO")
df.go <- data.frame(go)

# Finally, merge the above two data table and push it to Cytoscape.
# set DESCRIPTION as attribute data
nodeDataDefaults(g, attr = "DESCRIPTION") <- "undefined"
attr(nodeDataDefaults(g, attr = "DESCRIPTION"), "class") <- "STRING"

# set GENENAME as attribute data
nodeDataDefaults(g, attr = "name") <- "undefined"
attr(nodeDataDefaults(g, attr = "name"), "class") <- "STRING"

# set GO as attribute data
#nodeDataDefaults(g, attr = "go") <- "undefined"
#attr(nodeDataDefaults(g, attr = "go"), "class") <- "LIST"

# marge DESCRIPTION attribute.
for (index in 1:length(df.descriptions$DESCRIPTION)){
    nodeData(g, df.descriptions$ORF[[index]], "DESCRIPTION") <- df.descriptions$DESCRIPTION[[index]]
}

# marge GENENAME attribute.
for (index in 1:length(df.gene.names$GENENAME)){
    nodeData(g, df.gene.names$ORF[[index]], "name") <- df.gene.names$GENENAME[[index]]
}

# marge GO attribute.
#for (index in 1:length(df.gene.names$GENENAME)){
#    nodeData(g, df.gene.names$ORF[[index]], "name") <- df.gene.names$GENENAME[[index]]
#}

# show it in cytescape
cw <- CytoscapeWindow('vignette', graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')

'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns


[1] "DESCRIPTION"
[1] "name"
[1] "label"


# Upload an edge attribute table

In [13]:
# add attribute:"edgeType" and the type:"STRING"
edgeDataDefaults(g, attr = "edgeType") <- "undefined"
attr(edgeDataDefaults(g, attr = "edgeType"), "class") <- "STRING"

# get EdgeList and type
gal.table.fromvec <- gal.table[,1]
gal.table.type <- gal.table[,2]
gal.table.tovec <-  gal.table[,3]

# add attribute value to the network
for (index in 1:length(gal.table.fromvec)){
    edgeData(g, gal.table.fromvec[[index]] ,gal.table.tovec[[index]], "edgeType") <- gal.table.type[[index]]
}
cw <- CytoscapeWindow('vignette', graph=g, overwrite=TRUE)
displayGraph (cw)
layoutNetwork (cw, layout.name='degree-circle')

[1] "DESCRIPTION"
[1] "name"
[1] "label"
[1] "edgeType"


# Select edges based on some edge attributes
---------------------------------------

### How to select edges based on their attributes?

First, we have to set attributes to edges. Then, we can select edges based on attributes.

### Method : selectEdges

#### Usage
selectEdges(obj, edge.names, preserve.current.selection=TRUE)

#### Arguments

- **obj** : a CytoscapeWindowClass object.  
- **edge.names** : a list of strings, the names of edges to select.  
- **preserve.current.selection** : a logical object.  




In [14]:
# clear selection
clearSelection (cw)

In [16]:
# selection Edges
vec = c()
for (index in 1:length(gal.table.type)){
    if(gal.table.type[[index]] == 'pp'){
        vec = c(vec, paste(gal.table.fromvec[[index]], '(pp)', gal.table.tovec[[index]]))
    }
}
selectEdges (cw, vec)

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](select_edge_image.jpeg)

# Hide these edges
---------------------------------------

If you want to hide selected edges, you can use this method.

### Method : hideSelectedEdges

#### Usage

hideSelectedEdges(obj)

#### Arguments

obj a CytoscapeWindowClass object.

In [9]:
# The code is only this!!
# TODO : this methods have some error. I can't do anything with this method.
hideSelectedEdges (cw)

NULL

# Select nodes based on some node attributes
---------------------------------------

### method : selectNodes

#### Usage
selectNodes(obj, node.names, preserve.current.selection=TRUE)

#### Arguments

- **obj** : a CytoscapeWindowClass object.
- **node.names** : a list of strings, the names of nodes to select.
- **preserve.current.selection** : a logical object.


In [18]:
# get attributes data
gal.node.attrs <- read.table('sampleData/galFiltered_node.attrs',stringsAsFactors=FALSE, skip=1)
head(gal.node.attrs)

V1,V2,V3
YKR026C,=,1
YGL122C,=,1
YMR146C,=,1
YDR429C,=,1
YFL017C,=,1
YOL123W,=,1


In [19]:
g <- initNodeAttribute (graph=g, attribute.name='moleculeType', attribute.type='char', default.value='undefined')

# get EdgeAttrs
gal.node = gal.node.attrs[[1]]
gal.attrs = gal.node.attrs[[3]]

for (index in 1:length(gal.node)){
    nodeData (g, gal.node, 'moleculeType') <- as.character(gal.attrs[[index]])
}

cw <- setGraph (cw, g)
displayGraph (cw)

[1] "DESCRIPTION"
[1] "name"
[1] "label"
[1] "moleculeType"
[1] "edgeType"


In [20]:
# clear selection
clearSelection (cw)

In [21]:
vac = c()
for (index in 1:length(gal.node)){
    if(gal.attrs[[index]] == '1'){
        vec = c(vec, gal.node[[index]])
    }
}

selectNodes(cw, vec)

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](select_nodes_image.jpeg)

# Hide these nodes
---------------------------------------
### Method : hideSelectedNodes

#### Usage

hideSelectedNodes(obj)

#### Arguments

obj : a CytoscapeWindowClass object.


In [22]:
# TODO : There are some error in this method. I can't do anything by this.
hideSelectedNodes(cw)

not yet implemented


# Apply a layout the the network that is not hidden
---------------------------------------
### Method : getLayoutNames, layoutNetwork

We use two method in this section. 'getLayoutNames' let us know available layoutNames and we can apply one of them by 'layoutNetwork' method.

#### 1. Usage
getLayoutNames(obj)

#### 1. Arguments
obj : a CytoscapeConnectionClass object.

#### 2. Usage
layoutNetwork(obj, layout.name= grid )

#### 2. Arguments
obj a CytoscapeWindowClass object.
layout.name a string, one of the values returned by getLayoutNames, ’grid’ by default.


In [23]:
# get available layout names
getLayoutNames(cw)

[1] "attribute-circle"      "stacked-node-layout"   "degree-circle"        
 [4] "circular"              "attributes-layout"     "kamada-kawai"         
 [7] "force-directed"        "grid"                  "hierarchical"         
[10] "fruchterman-rheingold" "isom"                  "force-directed-cl"

In [24]:
# execute layout that you want
layoutNetwork(cw, layout.name= 'isom')

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](isom_graph_image.jpeg)

In [25]:
# execute layout that you want
layoutNetwork(cw, layout.name= 'kamada-kawai')

Then, you can get the output,like the follwoing image, in cytoscape.

![cytoscape image](kamada_kawai_image.jpeg)

# Apply a visual style to this network
---------------------------------------

By default, Cytoscape displays nodes as pale red circles circles, edges as blue undecorated lines, selected nodes as yellow and selected edges as red. RCy3 provides an easy way to change these defaults. More interestingly, RCy3 provides easy programmatic access to the vizmapper, by means of which the size, shape and color of nodes and edges is determined by the data attributes you have attached to those nodes and edges.

## The methods
---------------------------------------

There are so many methods to do this.

#TODO : I have to remove following message when I push. I think this section will be similar to cookbook, so I will copy and modify with it.

At this point I do every bit of the visual style manually, but it is always the same.

That is because I do not know how to transfer a visual style from one session to the next yet.

It would be great if that was automated (I do it manually for one network and then it is automatic for the following ones that have the same exact attributes)'